In [1]:
import xarray as xr
import numpy as np

from pathlib import Path


In [2]:
# Open the MAST dataset

ds = xr.open_dataset("mast_magnetics_data_constant_time.nc")
ds

<xarray.Dataset> Size: 5GB
Dimensions:                                   (shot_id: 6102, time: 1000,
                                               b_field_pol_probe_ccbv_channel: 37,
                                               b_field_pol_probe_obr_channel: 18,
                                               b_field_pol_probe_obv_channel: 17,
                                               flux_loop_channel: 14,
                                               b_field_tor_probe_saddle_voltage_channel: 9)
Coordinates:
  * shot_id                                   (shot_id) int32 24kB 12378 ... ...
  * time                                      (time) float64 8kB -0.05 ... 0.45
  * b_field_pol_probe_ccbv_channel            (b_field_pol_probe_ccbv_channel) object 296B ...
  * b_field_pol_probe_obr_channel             (b_field_pol_probe_obr_channel) object 144B ...
  * b_field_pol_probe_obv_channel             (b_field_pol_probe_obv_channel) object 136B ...
  * flux_loop_channel                         (flux_loop_channel) object 112B ...
  * b_field_tor_probe_saddle_voltage_channel  (b_field_tor_probe_saddle_voltage_channel) object 72B ...
Data variables:
    b_field_pol_probe_ccbv_field              (shot_id, time, b_field_pol_probe_ccbv_channel) float64 2GB ...
    b_field_pol_probe_obr_field               (shot_id, time, b_field_pol_probe_obr_channel) float64 879MB ...
    b_field_pol_probe_obv_field               (shot_id, time, b_field_pol_probe_obv_channel) float64 830MB ...
    flux_loop_flux                            (shot_id, time, flux_loop_channel) float64 683MB ...
    b_field_tor_probe_saddle_voltage          (shot_id, time, b_field_tor_probe_saddle_voltage_channel) float64 439MB ...
    ip_ref                                    (shot_id, time) float64 49MB ...
    i_plasma                                  (shot_id, time) float64 49MB ...
    ip                                        (shot_id, time) float64 49MB ...

In [3]:
# Build a .npz dataset from the xarray `ip` variable
# Assumes `ds` is already defined in the notebook and contains a DataArray 'ip'
import os
import numpy as np

# extract ip DataArray
ip_da = ds['ip']  # xarray DataArray with dims (shot_id, time)

# Convert to numpy array: (n_shots, n_time)
data = ip_da.values

# If there are NaNs, replace them by the per-time mean (column mean)
if np.isnan(data).any():
    col_mean = np.nanmean(data, axis=0)
    # Where NaN, fill with corresponding column mean
    inds = np.where(np.isnan(data))
    if inds[0].size > 0:
        data[inds] = np.take(col_mean, inds[1])

# Ensure a numeric float dtype suitable for the model
data = data.astype(np.float32)

n_samples = data.shape[0]

# Create labels (no anomaly labels available): default to zeros
labels = np.zeros(n_samples, dtype=int)

# Create is_train boolean mask with a reproducible random split
val_ratio = 0.2
rng = np.random.default_rng(42)
is_train = np.ones(n_samples, dtype=bool)
n_val = max(1, int(n_samples * val_ratio))
val_idx = rng.choice(n_samples, size=n_val, replace=False)
is_train[val_idx] = False

# Prepare output path and save as .npz with keys expected by the app
#os.makedirs('data/uploaded', exist_ok=True)
filename = 'ip_dataset_from_notebook.npz'
#filepath = os.path.join('data/uploaded', filename)
filepath = filename

np.savez_compressed(filepath, data=data, labels=labels, is_train=is_train)

print(f"Saved dataset to: {filepath}")
print('data.shape =', data.shape)
print('labels.shape =', labels.shape)
print('n_train =', int(is_train.sum()), 'n_val =', int((~is_train).sum()))


Saved dataset to: ip_dataset_from_notebook.npz
data.shape = (6102, 1000)
labels.shape = (6102,)
n_train = 4882 n_val = 1220
